# MSc in Data Analytics
# Sentiment Analysis

Assessment Task
Students are advised to review and adhere to the submission requirements documented after the assessment task.  
In this continuous assessment, You are required to identify and carry out an analysis of a large dataset gleaned from the twitter API. Instructions for accessing the data can be found here  
https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/  
https://www.toptal.com/apache/apache-spark-streaming-twitter  
OR You may use the data held here:  
https://archive.org/details/twitterstream?sort=-publicdate  
You must collect at least 1 year's tweets on a topic, this data should be stored as requested below, and you are then required to analyse any change sentiment that occurs over the time period that you have selected.  
Following your analysis, you are then required to make a time series forecast of the sentiment at 1 week, 1 month and 3 months going forward. This forecast must be displayed as a dynamic dashboard.   


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
import json, os

In [ ]:
# Parameters

# Topic
topic = "vaccine"

# Read the directory and find the bz2 files containint the tweets
dataset_path = r'c:\tmp\twitter'


# Size of the batch 
num_files = 100

destination_folder = f'c:\\\\tmp\\twitter-result\\'
#destination_folder = f'/twitter/'

In [ ]:
# Create the Spark Context
# fix to manage the error 
# "Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sba22243-step1").config("spark.sql.debug.maxToStringFields", 100).getOrCreate()
sqlContext = spark._wrapped

In [ ]:
def filter_tweets(filename):

    file_list = [] # this list contain the filenames of the dataset stored in the dataset_path folder
    # explore the subfolder
    for subdir, dirs, files in os.walk(dataset_path):
        for file in files:
            # if the file is a bz2 then get the filename
            if file.endswith('.bz2'):
                filename_bz2 = os.path.join(subdir, file)
                filename_bz2 = filename_bz2.replace('\\', '\\\\')
                file_list.append(os.path.join(subdir, file))
    print(f'Num of files {len(file_list)}')
    #print(file_list)
    
    # Read a batch of files in parallel
    for i in range(0, len(file_list), num_files):
        sub_list = file_list[i:i+num_files]
        dataframe = sqlContext.read.json(sub_list)

        dataframe_result = dataframe.filter((col('lang') == 'en') & col('text').rlike(f'(?i){topic}'))
        dataframe_result = dataframe_result.drop('entities', 'extended_entities', 'retweeted_status')

        print(f'Baatch {i} - writing {dataframe_result.count()} tweets')
        dataframe_result.write.mode("append").json('file:///' + destination_folder + filename + '.json')
        #dataframe_result.write.mode("append").json('hdfs://hadoop.local/' + destination_folder + filename + '.json')


In [ ]:
filter_tweets('2021-january')